In [7]:
import together
import os
import tqdm
import smart_open
from dolma.core.paths import glob_path
import random
import json
import copy
import openai

random.seed(42)
# client = together.Together(api_key=os.environ.get("TOGETHER_API_KEY"))
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
base_path = "s3://ai2-llm/pretraining-data/sources/cccc/v2/documents/*/*.gz"
all_paths = list(glob_path(base_path))
random.shuffle(all_paths)

# response = client.chat.completions.create(
#     model="mistralai/Mixtral-8x7B-Instruct-v0.1",
#     messages=[{"role": "user", "content": "tell me about new york"}],
# )
# print(response.choices[0].message.content)

In [2]:
prompt = """
Given the following HTML snippet enclosed in ```quotes```, respond TRUE if the Creative Common license in it refers to an image, FALSE if it refers to text, and UNKNOWN if you can't determine the type of content the license refers to.

```html
{snippet}
```
"""

In [3]:
# Grabbing documents
documents = []
with smart_open.open(all_paths[0]) as f:
    for line in tqdm.tqdm(f):
        documents.append(json.loads(line))
        if len(documents) % 100 == 0:
            break

99it [00:00, 469.01it/s]


In [5]:
!pip install openai

  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 10.8 MB/s eta 0:00:00
Using cached anyio-4.4.0-py3-none-any.whl (86 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [4]:
responses = []
# model_name = "meta-llama/Llama-3-70b-chat-hf"
model_name = "gpt-4o"

for doc in tqdm.tqdm(documents):
    extracted_licenses = {
        license_name: license_snippet[0].strip()
        for license_name, license_snippet in doc["metadata"]["attribute_spans"].items()
        if license_name != "copyright" and isinstance(license_snippet, list)
    }

    labeled_licenses = {}
    for license_name, license_snippet in extracted_licenses.items():
        response = client.chat.completions.create(
            model=model_name,
            temperature=0.2,
            messages=[{"role": "user", "content": prompt.format(snippet=license_snippet)}],
        )
        labeled_licenses[license_name] = {
            "label": response.choices[0].message.content,
            "snippet": license_snippet,
            "model": model_name,
        }

    doc_with_labels = copy.deepcopy(doc)
    doc_with_labels["metadata"]["labeled_licenses"] = labeled_licenses
    responses.append(doc_with_labels)

 34%|███▍      | 34/100 [02:25<04:41,  4.26s/it]


KeyboardInterrupt: 

In [ ]:
doc_with_labels

{'added': '2024-06-03T09:32:28.451Z',
 'created': '2014-08-01T03:41:42.000Z',
 'id': 'ry4bjpowjg47ivpwweaeqy323oq5k5wn',
 'metadata': {'attribute_spans': {'cc_re__cc_re__cc_by_sa_3_0_us': ['it\' href=\'//www.blogger.com/rearrange?blogID=2029263360628848260&widgetType=HTML&widgetId=HTML7&action=editWidget&sectionId=col-bottom\' onclick=\'return _WidgetManager._PopupConfig(document.getElementById("HTML7"));\' target=\'configHTML7\' title=\'Edit\'>\n<img alt=\'\' height=\'18\' src=\'http://img1.blogblog.com/img/icon18_wrench_allbkg.png\' width=\'18\'/>\n</a>\n</span>\n</span>\n<div class=\'clear\'></div>\n</div><div class=\'widget HTML\' id=\'HTML9\'>\n<h2 class=\'title\'>copyright</h2>\n<div class=\'widget-content\'>\n<a href="http://creativecommons.org/licenses/by-sa/3.0/us/" rel="license"><img alt="Creative Commons License" style="border-width:0" src="http://creativecommons.org/images/public/somerights20.png"/></a><br/>This work is licensed under a <a href="http://creativecommons.org/l